# Библиотеки

In [1]:
import neptune
import pandas as pd

from cv import get_indices
from load_data import load
from model_selection import training
import random
from NN import simple_torchpl
from pl_framework import nn_training
from pytorch_forecasting.metrics import MAPE
import torch

In [2]:
neptune.init('iliaavilov/SIBUR')

Project(iliaavilov/SIBUR)

In [3]:
import os
os.environ['NEPTUNE_API_TOKEN']="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiYzI0ZTMzMDQtZTFmZi00ZjgxLWI4NGQtZGJiOWQyNDJiYjE5In0="
os.environ['NEPTUNE_PROJECT']="iliaavilov/SIBUR"
os.environ['NEPTUNE_NOTEBOOK_ID']="3334104b-e4da-40ca-a353-d78790164c0b"
os.environ['NEPTUNE_NOTEBOOK_PATH']="Desktop/Projects/sibur_2020/modeling.ipynb"

# Константы

In [4]:
data_path = 'data/'
random_state = 54321

# Загрузка данных

In [5]:
train_features, train_targets, _ = load(data_path)

# Валидация

In [6]:
cv = get_indices(train_targets, [(pd.to_datetime('2020-03-01 00:00:00'), pd.to_datetime('2020-03-15 00:00:00')),
                                 (pd.to_datetime('2020-03-15 00:00:00'), pd.to_datetime('2020-03-31 00:00:00')),
                                 (pd.to_datetime('2020-04-15 00:00:00'), pd.to_datetime('2020-04-30 00:00:00'))
                                ]
                )

In [7]:
#train_targets = train_targets.drop('timestamp', axis = 'columns')
#train_features = train_features.drop('timestamp', axis = 'columns')

# Подбор модели

In [ ]:
my_training = training(name = 'LGBM_C2H6', nn_model = simple_torchpl, training_nn = nn_training,
                       description = 'Ffill. 1 train set for all folds. 1 model for all targets. LGBM. \
                       Loss = MAPE. No normalisation. Only first 2 fold and test set(last fold).Dropped data before 2020-02-15 00:00:00',
                       upload_source_files = ['cv.py', 
                                              'load_data.py', 
                                              'model_selection.py',
                                              'NN.py',
                                              'pl_framework.py'])
my_training.set_up_studying(random_state = random_state)

model = 'lgbm'

def params_func(trial, X):
    return(
{        'objective': 'regression',
        'metric': 'mape',
        'n_jobs': -1,
        'n_estimators': 10,
        'random_state': random_state,
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.5),
        'min_child_samples': trial.suggest_int('min_child_samples', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'n_in': trial.suggest_int('n_in', 1, 150),
        'n_back': trial.suggest_int('n_back', 1, 150)}
    )

n_trials = 100
my_training.train(X = train_features, 
                  y = train_targets, 
                  cv = cv, 
                  model=model, 
                  params_func = params_func, 
                  n_trials = n_trials)

https://ui.neptune.ai/iliaavilov/SIBUR/e/SIB-248


[I 2020-11-29 17:38:40,283] A new study created in memory with name: no-name-d16e1a02-feef-4d12-b8cd-5751ea3095c4


current params: {'objective': 'regression', 'metric': 'mape', 'n_jobs': -1, 'n_estimators': 10, 'random_state': 54321, 'num_leaves': 83, 'learning_rate': 0.12858766698944454, 'min_child_samples': 98, 'feature_fraction': 0.8750823012386388, 'bagging_fraction': 0.6578708780010873, 'n_in': 11, 'n_back': 128}
X shape before dropped nans (3647, 11)
cv before trans: 719
Initial shape of X after dropping nans: (3519, 11)
Initial shape of y: (3647, 5)


In [ ]:
#neptune.log_metric('leaderboard_mape', 3.2443)
#neptune.project.get_experiments('SIB-205')[0].stop()
#neptune.stop()